In [ ]:
# Building a company brochure generator with gradio

In [ ]:
# imports

import os
import requests
from IPython.display import Markdown, display
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI, AzureOpenAI
import google.generativeai
import anthropic

# DEEPSEEK AZURE OPENAI
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

In [ ]:
import gradio as gr # oh yeah!

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')


# personalise env data FOR AZURE OPENAI
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_URL")
azure_openai_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini")
azure_openai_subscription_key = os.getenv("AZURE_OPENAI_API_KEY")

# personalise env data FOR DEEPSEEK AZURE OPENAI
azure_deepseek_endpoint = os.getenv("DEEPSEEK_AZURE_INFERENCE_SDK_ENDPOINT")
azure_openai_deepseek_endpoint=f"{azure_deepseek_endpoint}/models"
azure_deepseek_deployment= os.getenv("DEPLOYMENT_NAME", "DeepSeek-V3-0324")
azure_deepseek_subscription_key = os.getenv("DEEPSEEK_AZURE_INFERENCE_SDK_KEY")

# personalise env data FOR GROK AZURE OPENAI
azure_grok_endpoint = os.getenv("GROK_AZURE_INFERENCE_SDK_ENDPOINT")
azure_grok_deployment = os.getenv("GROK_DEPLOYMENT_NAME", "grok-3-mini")
azure_grok_subscription_key = os.getenv("GROK_AZURE_INFERENCE_SDK_KEY")



if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OPENAI API Key not set")
if azure_openai_subscription_key:
    print(f"AZURE OpenAI API Key exists and begins {azure_openai_subscription_key[:8]}")
else:
    print("AZURE OPENAI API Key not set")

if azure_deepseek_subscription_key:
    print(f"DEEPSEEK AZURE API Key exists and begins {azure_deepseek_subscription_key[:7]}")
else:
    print("DEEPSEEK AZURE API Key not set")

if azure_grok_subscription_key:
    print(f"GROK_AZURE API Key exists and begins {azure_grok_subscription_key[:8]}")
else:
    print("GROK_AZURE API Key not set")

In [ ]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()

# claude = anthropic.Anthropic()

# google.generativeai.configure()

# GPT AZURE OPENAIS
# Initialize Azure OpenAI client with key-based authentication
clientGPTAzure = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_subscription_key,
    api_version="2025-01-01-preview",
)

# Initialize Azure OpenAI for Deepseek client with key-based authentication
# api_version="2025-01-01-preview",
clientDeepseekAzureOpenAI = AzureOpenAI(
    azure_endpoint=azure_deepseek_endpoint,
    api_key=azure_deepseek_subscription_key, 
    api_version="2024-10-21",
)



# DEEPSEEK AZURE OPENAI
clientDeepSeekAzure = ChatCompletionsClient(endpoint=azure_openai_deepseek_endpoint, credential=AzureKeyCredential(azure_deepseek_subscription_key))

# GROK AZURE OPENAI
clientGrokAzure = ChatCompletionsClient(endpoint=azure_grok_endpoint, credential=AzureKeyCredential(azure_grok_subscription_key))



In [ ]:

# A generic system message - no more snarky adversarial AIs!

system_message = "You are a helpful assistant"

In [ ]:
# Let's wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [ ]:
# Let's use Markdown
# Are you wondering why it makes any difference to set system_message when it's not referred to in the code below it?
# I'm taking advantage of system_message being a global variable, used back in the message_gpt function (go take a look)
# Not a great software engineering practice, but quite common during Jupyter Lab R&D!

system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [ ]:

def stream_openai_gpt(prompt):
    # stream = openai.chat.completions.create(
    #     model='gpt-4o-mini',
    #     messages=messages,
    #     stream=True
    # )
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    
    stream = clientGPTAzure.chat.completions.create(
    model=azure_openai_deployment,
    messages=messages,
    max_tokens=6553,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=True
    )

    result = ""
    for chunk in stream:
        if not chunk.choices: 
            continue
        
        delta = chunk.choices[0].delta
        if delta and getattr(delta, "content", None):  
            content = delta.content
            result += content
            yield result

In [ ]:
view_one = gr.Interface(
    fn=stream_openai_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view_one.launch()

In [ ]:
def stream_deepseek(prompt):
    system_message = "You are a helpful assistant that responds in plain text"
    sysContent=system_message
    usrContent=prompt

    result = clientDeepSeekAzure.complete(
    messages=[
    SystemMessage(content=sysContent),
    UserMessage(content=usrContent)
    ],
    model = azure_deepseek_deployment,
    max_tokens=1000,
        stream=True
    )

    
    response = ""
    for chunk in result:
        if not chunk.choices:  # <-- skip empty events
            continue
        
        delta = chunk.choices[0].delta
        if delta and getattr(delta, "content", None):  # safe check
            content = delta.content
            response += content
            yield response
    # print(response.choices[0].message.content)

In [ ]:
view = gr.Interface(
    fn=stream_deepseek,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [ ]:
def stream_deepseek_openai(prompt):
    system_message = "You are a helpful assistant that responds in Markdown"
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    
    stream = clientDeepseekAzureOpenAI.chat.completions.create(
    model=azure_deepseek_deployment,
    messages=messages,
    max_tokens=2048,
    temperature=0.8,
    top_p=0.1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=True
    )

    # response = clientDeepseekAzureOpenAI.chat.completions.create(
    # model=azure_deepseek_deployment, # Replace with your model dpeloyment name.
    # messages=[
    #     {"role": "system", "content": "You are a helpful assistant."},
    #     {"role": "user", "content": "Explain Riemann's conjecture in 1 paragraph"}
    # ],
    # )
    # print(response.model_dump_json(indent=2))
    # print(stream.model_dump_json(indent=2))
    # print(stream.choices[0].model_dump_json(indent=2))
    result = ""
    for chunk in stream:
        if not chunk.choices:  
            continue
        delta = chunk.choices[0].delta
        if delta and getattr(delta, "content", None): 
            content = delta.content
            result += content
            yield result

In [ ]:
view = gr.Interface(
    fn=stream_deepseek_openai,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [ ]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
# With massive thanks to Bill G. who noticed that a prior version of this had a bug! Now fixed.

system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [ ]:
def stream_brochure(company_name, url, model):
    yield ""
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    
    if model=="AZURE_OPENAI_GPT":
        result = stream_openai_gpt(prompt)
    elif model=="OPENAI_DEEPSEEK":
        result = stream_deepseek_openai(prompt)
    elif model=="AZURE_OPENAI_DEEPSEEK":
        result = stream_deepseek(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["AZURE_OPENAI_GPT", "OPENAI_DEEPSEEK","AZURE_OPENAI_DEEPSEEK"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()